In [1]:
# 去除爬虫功能
def remove_spider_data(single_log):
    # 定义排除爬虫规则集
    exclude_set = [
        'AhrefsBot',
        'archive.org_bot',
        'baiduspider',
        'Baiduspider',
        'bingbot',
        'DeuSu',
        'DotBot',
        'Googlebot',
        'iaskspider',
        'MJ12bot',
        'msnbot',
        'Slurp',
        'Sogou web spider',
        'Sogou Push Spider',
        'SputnikBot',
        'Yahoo! Slurp China',
        'Yahoo! Slurp',
        'YisouSpider',
        'YodaoBot',
        'bot.html'
    ]
    count = 0  # 初始计数用于计算日志中是否包含爬虫
    for spider in exclude_set:  # 循环读取每个爬虫
        if single_log.find(spider) != -1:  # 如果爬虫出现在日志里面
            count += 1  # count + 1
    if count > 0:  # 如果结果不为0，意味着日志中有爬虫
        return 0  # 返回 0
    else:  # 否则
        return 1  # 返回1


# 读取日志数据
def get_raw_log(file):
    fn_read = open(file, 'r')  # 打开要读取的日志文件对象
    content = fn_read.readlines()  # 以列表形式读取日志数据
    fn_read.close()  # 关闭文件对象
    for single_log in content:  # 循环判断每天记录
        rule1 = single_log.find('GET /__ua.gif?') != -1  # 定义日志规则：含ua.gif的请求
        rule2 = remove_spider_data(single_log)
        if rule1 == True and rule2 == True:  # 如果同时符合2条规则，则执行
            fn_write = open('ua_data.txt', 'a+')  # 打开要保存的ua日志文件对象
            fn_write.writelines(single_log)  # 写入符合规则的日志
            fn_write.close()  # 关闭文件对象


# 解析每条日志数据
def split_ua(line):
    import re
    # 定义不同日志分割的正则表达式
    ip_rule = r'[\d.]*'  # 定义IP规则，例如203.208.60.230
    time_rule = r'\[[^\[\]]*\]'  # 定义时间规则，例如[02/Mar/2016:14:00:23 +0800]
    request_rule = r'\"[^\"]*\"'  # 定义请求规则
    status_rule = r'\d+'  # 定义返回的状态码规则，例如200
    bytes_rule = r'\d+'  # 返回的字节数，例如326
    refer_rule = r'\"[^\"]*\"'  # 定义refer规则
    user_agent_rule = r'\"[^\"]*\"'  # 定义user agnet规则
    # 原理：主要通过空格和-来区分各不同项目，各项目内部写各自的匹配表达式
    log_re_pattern = re.compile(r'(%s)\ -\ -\ (%s)\ (%s)\ (%s)\ (%s)\ (%s)\ (%s)' % (
        ip_rule, time_rule, request_rule, status_rule, bytes_rule, refer_rule, user_agent_rule), re.VERBOSE)  # 完整表达式模式
    matchs = log_re_pattern.match(line)  # 匹配
    if matchs != None:  # 如果不为空
        allGroups = matchs.groups()  # 获得所有匹配的列表
        ip_info = allGroups[0]  # IP信息
        time_info = allGroups[1]  # 时间信息
        request_info = allGroups[2]  # 请求信息
        status_info = allGroups[3]  # 状态信息
        bytes_info = allGroups[4]  # 字节数信息
        refer_info = allGroups[5]  # refer信息
        user_agent_info = allGroups[6]  # user agent信息
        return ip_info, time_info, request_info, status_info, bytes_info, refer_info, user_agent_info


# 主程序
def get_final_data():
    file = 'traffic_log_for_dataivy'  # 定义原始日志
    get_raw_log(file)  # 读取非结构化文本数据
    fn_r = open('ua_data.txt', 'r')  # 打开要读取的ua日志文件
    content = fn_r.readlines()  # 读取ua所有日志记录
    fn_r.close()  # 关闭ua文件对象
    fn_w = open('final_data.txt', 'a+')  # 打开要写入的格式化文件
    for line in content:  # 按行循环
        ip_info, time_info, request_info, status_info, bytes_info, refer_info, user_agent_info = split_ua(
            line)  # 获得分割后的数据
        log_line = ip_info + '!' + time_info + '!' + request_info + '!' + status_info + '!' + bytes_info + '!' + refer_info \
                   + '!' + user_agent_info  # 按指定规则组合用于写入文件
        fn_w.writelines(log_line)  # 写入文件
        fn_w.writelines('\n')  # 写入换行符用于换行
    fn_w.close()  # 关闭写入的文件对象


# 执行程序
get_final_data()  # 执行所有程序